In [170]:
import os
import pandas as pd
import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, Table, select
from sklearn import model_selection


# Load Data

In [197]:
def export_db_to_df(db_path):
    '''To export data from db to dataframe'''
    # Check if the database files exist
    if not os.path.exists(db_path):
        print("Database files not found. Please make sure they are in the 'data' folder.")
        exit(1)

    # Create SQLAlchemy engine for database
    engine = db.create_engine(f"sqlite:///{db_path}")
    connection = engine.connect()

    # Create SQLAlchemy MetaData objects
    metadata = db.MetaData()
    metadata.reflect(bind=engine)

    # Print table names
    table_name = list(metadata.tables.keys())[0]
    print("Tables in database:")
    print(table_name)


    # Query entire table and export into df
    db_table = Table(table_name, metadata, autoload=True, autoload_with=engine)
    query = db.select(db_table)
    result = connection.execute(query)
    df = pd.DataFrame(result.fetchall(), columns=db_table.columns.keys())
    df = df.reset_index(drop=True)
    print("Complete export to dataframe")

    return df


In [198]:
# Paths to the database files
pre_purchase_db_path = r"C:\Users\anna\Documents\gitHub\NTU\InterpolationModel\data\cruise_pre.db"
post_trip_db_path = r"C:\Users\anna\Documents\gitHub\NTU\InterpolationModel\data\cruise_post.db"

pre_cruise = export_db_to_df(pre_purchase_db_path)
post_cruise = export_db_to_df(post_trip_db_path)

Tables in database:
cruise_pre
Complete export to dataframe
Tables in database:
cruise_post
Complete export to dataframe


# EDA

In [180]:
def check_each_features(df):
    for feature in df.columns:
        print("count for each unique values:")
        print(df[feature].value_counts())
        print("no. of missing data:", len(df[df[feature].isna()]))
        print("percentage of missing data:", str(round(len(df[df[feature].isna()])/len(df)*100, 2)) + "%", "\n")

In [167]:
check_each_features(pre_cruise)

count for each unique values:
0         1
109280    1
23262     1
17117     1
19164     1
         ..
21792     1
109855    1
107806    1
113949    1
2047      1
Name: index, Length: 133746, dtype: int64
no. of missing data: 0
percentage of missing data: 0.0% 

count for each unique values:
Female    60950
Male      59340
Name: Gender, dtype: int64
no. of missing data: 13456
percentage of missing data: 10.06% 

count for each unique values:
03/05/1984    22
21/04/1998    21
13/04/1981    21
22/02/1998    21
02/12/1984    21
              ..
24/06/1951     1
1837-10-09     1
07/06/1947     1
1856-01-21     1
24/04/2014     1
Name: Date of Birth, Length: 25746, dtype: int64
no. of missing data: 14684
percentage of missing data: 10.98% 

count for each unique values:
Direct - Email Marketing    54645
Direct - Company Website    54591
Indirect - Search Engine    12288
Indirect - Social Media     12222
Name: Source of Traffic, dtype: int64
no. of missing data: 0
percentage of missing data: 

In [175]:
check_each_features(post_cruise)

count for each unique values:
0         1
109280    1
23262     1
17117     1
19164     1
         ..
21792     1
109855    1
107806    1
113949    1
2047      1
Name: index, Length: 133746, dtype: int64
no. of missing data: 0
percentage of missing data: 0.0% 

count for each unique values:
Blastoise    69151
Lapras       31057
blast         3046
lapras        2955
blastoise     2917
blast0ise     2906
lap           2898
IAPRAS        2885
Name: Cruise Name, dtype: int64
no. of missing data: 15931
percentage of missing data: 11.91% 

count for each unique values:
Luxury      54414
Standard    51011
Deluxe       8260
Name: Ticket Type, dtype: int64
no. of missing data: 20061
percentage of missing data: 15.0% 

count for each unique values:
337 KM        669
594 KM        408
862 KM        362
404 KM        359
2475 KM       357
             ... 
-448 Miles      1
-2637 KM        1
-3786 KM        1
-1714 KM        1
-2132 KM        1
Name: Cruise Distance, Length: 8995, dtype: int64
no.

In [199]:
# Date of birth is dropped 
pre_cruise = pre_cruise.drop(["Date of Birth", "index", "Ext_Intcode", "Logging"], axis = 1)

In [200]:
post_cruise = post_cruise.drop(["index", "Ext_Intcode"], axis = 1)

In [201]:
combined_data = pd.concat([pre_cruise, post_cruise], axis = 1)

In [202]:
combined_data = combined_data.dropna(subset=["Ticket Type"])

In [204]:
combined_data = combined_data.fillna("Missing")

In [186]:
y = combined_data["Ticket Type"]

In [189]:
combined_data = combined_data.drop(["Ticket Type"], axis = 1)

In [203]:
check_each_features(combined_data)

count for each unique values:
Female    51805
Male      50479
Name: Gender, dtype: int64
no. of missing data: 11401
percentage of missing data: 10.03% 

count for each unique values:
Direct - Email Marketing    46506
Direct - Company Website    46267
Indirect - Search Engine    10474
Indirect - Social Media     10438
Name: Source of Traffic, dtype: int64
no. of missing data: 0
percentage of missing data: 0.0% 

count for each unique values:
A little important      24931
Somewhat important      24815
Very important          19088
Not at all important    17289
Extremely important     11031
Name: Onboard Wifi Service, dtype: int64
no. of missing data: 16531
percentage of missing data: 14.54% 

count for each unique values:
4.0    24637
5.0    21629
3.0    17326
2.0    16662
1.0    15028
0.0     5120
Name: Embarkation/Disembarkation time convenient, dtype: int64
no. of missing data: 13283
percentage of missing data: 11.68% 

count for each unique values:
3.0    23011
2.0    22587
4.0    18

In [196]:
check_each_features(y)

AttributeError: 'Series' object has no attribute 'columns'

In [185]:
combined_data.columns

Index(['Gender', 'Source of Traffic', 'Onboard Wifi Service',
       'Embarkation/Disembarkation time convenient', 'Ease of Online booking',
       'Gate location', 'Onboard Dining Service', 'Online Check-in',
       'Cabin Comfort', 'Onboard Entertainment', 'Cabin service',
       'Baggage handling', 'Port Check-in Service', 'Onboard Service',
       'Cleanliness', 'Cruise Name', 'Ticket Type', 'Cruise Distance', 'WiFi',
       'Dining', 'Entertainment'],
      dtype='object')

In [188]:
categorical_column = [
                    "Gender", 
                    "Source of Traffic"
                    "Onboard Wifi Service",
                    "Cruise Name",
                    "Cruise Distance" ]

numerical_column = [
                    "Embarkation/Disembarkation time convenient", 
                    "Ease of Online booking", 
                    "Gate location", 
                    "Onboard Dining Service", 
                    "Online Check-in",
                    "Cabin Comfort",
                    "Onboard Entertainment",
                    "Cabin service",
                    "Baggage handling",
                    "Port Check-in Service",
                    "Onboard Service",
                    "Cleanliness",
                    "WiFi",
                    "Dining",
                    "Entertainment"
                    ]

In [195]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(combined_data, y, test_size = 0.2, stratify=y, random_state = 2023)
